<a href="https://colab.research.google.com/github/Coralain10/ML_Archaeological-Objects-Reconstruction/blob/main/KerasSimpleModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, LSTM
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import math

In [2]:
!gdown 1AIQPe2G6ZHPwZLmo19-ewbqCNT7TDfYE

Downloading...
From: https://drive.google.com/uc?id=1AIQPe2G6ZHPwZLmo19-ewbqCNT7TDfYE
To: /content/FullAndFracture.npy
100% 466M/466M [00:02<00:00, 189MB/s]


In [3]:
data = np.load('FullAndFracture.npy', allow_pickle=True)

In [4]:
data = np.array(data, dtype = np.int32)

In [5]:
X = data[1]
Y = data[0]

In [6]:
lx = X.shape[0]
ly = Y.shape[0]

In [7]:
X = list(X)
Y = list(Y)

In [8]:
for i in range(lx):
  X[i] = X[i].flatten()
for i in range(ly):
  Y[i] = Y[i].flatten()

In [9]:
X, Y = np.array(X), np.array(Y)

In [10]:
x_train, x_val, y_train, y_val = train_test_split(X, Y, train_size=0.7)

In [11]:
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(4978, 32768) (4978, 32768)
(2134, 32768) (2134, 32768)


In [12]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(32768,)))
model.add(tf.keras.layers.Dense(768, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(768, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(32768))

model.compile(optimizer = 'adam',
         loss = 'mean_squared_error',
         metrics = ['mse'])

model.fit(x_train, y_train, epochs = 200)

Epoch 1/200
156/156 [==============================] - 92s 587ms/step - loss: 0.0351 - mse: 0.0351
Epoch 2/200
156/156 [==============================] - 91s 580ms/step - loss: 0.0214 - mse: 0.0214
Epoch 3/200
156/156 [==============================] - 92s 587ms/step - loss: 0.0152 - mse: 0.0152
Epoch 4/200
156/156 [==============================] - 92s 586ms/step - loss: 0.0117 - mse: 0.0117
Epoch 5/200
156/156 [==============================] - 92s 587ms/step - loss: 0.0099 - mse: 0.0099
Epoch 6/200
156/156 [==============================] - 90s 578ms/step - loss: 0.0090 - mse: 0.0090
Epoch 7/200
156/156 [==============================] - 89s 568ms/step - loss: 0.0080 - mse: 0.0080
Epoch 8/200
156/156 [==============================] - 88s 564ms/step - loss: 0.0071 - mse: 0.0071
Epoch 9/200
156/156 [==============================] - 89s 572ms/step - loss: 0.0066 - mse: 0.0066
Epoch 10/200
156/156 [==============================] - 89s 570ms/step - loss: 0.0067 - mse: 0.0067
Epoch 11/

In [31]:
prediction = model.predict(x_val)

67/67 [==============================] - 7s 100ms/step


In [28]:
prediction = np.array(np.around(prediction[0]), dtype=np.int32)

In [29]:
patch = []
for i in range(prediction.shape[0]):
  if prediction[i] == x_val[1][i]:
    patch.append(0)
  else:
    patch.append(1)
patch = np.array(patch)

In [16]:
def plot3d(verts, s=10, c=(105,127,155), show_grid=False):
    x, y, z = zip(*verts)
    color = f'rgb({c[0]}, {c[1]}, {c[2]})'
    trace = go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(
            size=s,
            color=color,
            line=dict(
                color='rgba(217, 217, 217, 0.14)',
                width=0.5
            ),
            opacity=1
        )
    )
    data = [trace]
    layout = go.Layout(
        margin=dict(l=0, r=0, b=0, t=0),
        scene = go.Scene(
            xaxis=dict(visible=show_grid),
            yaxis=dict(visible=show_grid),
            zaxis=dict(visible=show_grid)
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [17]:
def point_cloud_to_volume(points, vsize, radius=1.0):
    vol = np.zeros((vsize,vsize,vsize), dtype=np.bool)
    voxel = 2*radius/float(vsize)
    locations = (points + radius)/voxel
    locations = locations.astype(int)
    vol[locations[:,0],locations[:,1],locations[:,2]] = 1.0
    
    return vol

In [18]:
def volume_to_point_cloud(vol):
    """ vol is occupancy grid (value = 0 or 1) of size vsize*vsize*vsize
        return Nx3 numpy array.
    """
    vsize = vol.shape[0]
    assert(vol.shape[1] == vsize and vol.shape[1] == vsize)
    points = []
    for a in range(vsize):
        for b in range(vsize):
            for c in range(vsize):
                if vol[a,b,c] == 1:
                    points.append(np.array([a,b,c]))
    if len(points) == 0:
        return np.zeros((0,3))
    points = np.vstack(points)
    
    return points

def auto_pcl_to_volume(points, vsize):
	data_min = np.min(points)
	data_max = np.max(points)
	radius = max(abs(data_min), data_max)
	radius = math.ceil(radius*100) / 100
	vol = point_cloud_to_volume(points, vsize, radius)

	return vol

In [19]:
def plot_vol(vol, s=10, c=(105,127,155), show_grid=False):
    if vol.dtype != np.bool:
        vol = vol > 0

    pc = volume_to_point_cloud(vol)
    plot3d(pc, s, c, show_grid)

In [30]:
plot_vol(np.reshape(patch, (32,32,32)))
plot_vol(np.reshape(x_val[1],(32,32,32)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [32]:
model.save('saved_model')